In [2]:
from geopy.geocoders import Nominatim, OpenCage
import os
import pandas as pd
from time import sleep

# Geocoding

In [3]:
os.chdir('../')

#geolocator = Nominatim(user_agent="get-institute-location")
geolocator = OpenCage(api_key='017db8a44898418985a8f930f8d224ab')

df = pd.read_csv('public/Institutions - Top paper - CS.csv',on_bad_lines='warn')

locations = []

#Iterate through institute names and get their location
for idx, row in df.iterrows():
    search_string = f"{row['Institutions']}, {row['Countries/Regions']}"
    try: 
        location = geolocator.geocode(search_string)
        if location is not None:
            locations.append({
                'Institution': row['Institutions'],
                'Country': row['Countries/Regions'],
                'Latitude': location.latitude,
                'Longitude': location.longitude
            })
        else:
            locations.append({
                'Institution': row['Institutions'],
                'Country': row['Countries/Regions'],
                'Latitude': None,
                'Longitude': None
            })
        print(search_string, ":::", location)
    except Exception as e:
        print(f"Error geocoding {search_string} : {e}")
    sleep(1.5) #Respect API limits

CHINESE ACADEMY OF SCIENCES, CHINA ::: 100824 China, Beijing, Yuetan Subdistrict, Sanlihe Road 52, Chinese Academy of Sciences
UNIVERSITY OF CALIFORNIA SYSTEM, USA ::: University of California System Administration Central Office, Franklin Street, Oakland, CA 94616, United States of America
TSINGHUA UNIVERSITY, CHINA ::: 100084 China, Beijing, Haidian District, 东升镇, Shuangqing Road 30, Tsinghua University
SOUTHEAST UNIVERSITY - CHINA, CHINA ::: China
XIDIAN UNIVERSITY, CHINA ::: 710126 China, Shaanxi, Chang'an District, 南广场 266号, Xidian University (South Campus)
UNIVERSITY OF ELECTRONIC SCIENCE & TECHNOLOGY OF CHINA, CHINA ::: 610054 China, Sichuan, Chengdu, Mengzhuiwan, Section 1, First Ring Road East, Jianshe North Road
NANYANG TECHNOLOGICAL UNIVERSITY, SINGAPORE ::: Nanyang Technological University, 50 Nanyang Avenue, Singapore 639798, Singapore
CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE (CNRS), FRANCE ::: Avenue de la Recherche Scientifique, 45100 Orléans, France
SWISS FEDERAL IN

# Save Results

In [4]:
location_dict = {loc['Institution']: loc for loc in locations}

original_df = pd.read_csv('public/Institutions - Top paper - CS.csv', on_bad_lines='warn')

original_df['Latitude'] = original_df['Institutions'].map(lambda x: location_dict.get(x, {}).get('Latitude'))
original_df['Longitude'] = original_df['Institutions'].map(lambda x: location_dict.get(x, {}).get('Longitude'))

original_df.to_csv('public/Institutions location.csv', index=False)